# MOBIDEV 
## *Test task for Data Scientist position*
### candidate: Dmitri Kochubei
### email: dmitri.kochubei@gmail.com
___
1. *Provide with useful insights from the data* - 

This part requires further investigation, however there is a problem with, I beleive, 607 entries in the training dataset. review_scores are missing for those entries.
___
2. *Find the top impact factors on the price in the data* - 

Full list of top impact factors can be viewed at the very buttom of this notebook. But here is the list of the most impactful features based on Gradient Boosting Regressor model:

    accommodates - max # of people to reside 
    bathrooms
    cleaning_fee
    beds 
    description_beds 
    number_of_reviews
    host_acceptance_rate
    rm_tp_entirehomeapt - room type (Entire Apartment)

There are no surprises in these features, They can be broken down into 2 categories:
    
    description of a place (size, type, amenities)
    description of a host 
    
So the price mostly depends on the qualities of the apartment and renter's behavior.
___

3. *Select the best metric for the model evaluation and justify the choice* - 
    
    For now the stanard MSE is chosen, as the most commonly used criteria for regression models. The choice may change in the future, after more careful evaluation.
___    
5. *Describe the business value of your results* - 

I'm not really familiar with the context of the dataset, so it's hard to think of specific applications. But let's say, a host on the platform undervalues or overvalues her listing (given that the host has a choice). Using ML model the platform could nudge that host to adjust the price for higher booking rate.
____
*Final thoughts* - 

There is a lot more to be done with this dataset:
    
    MUCH more EDA, there is hardly anything done due to time limits
    Model tuning
    ...
    
But I had to stop somewhere.


If the dataset is big enough I would try deep learning.
___

# Logic of this notebook
1. A number of functions is defiend to pre-process and or encode all the features.
2. 2 Datasest (training and testing) are compiled for modeling.
3. A number of Models is tested.
4. Resulting predictions are generated using the best model.

In [19]:
import pandas as pd
import numpy as np
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt

# Pre-Processed dataset 

In [20]:
columns = [
'summary', # text tfidf
'description', # text tfidf
'neighborhood_overview', # text tfidf
'transit', # text tfidf
'host_about', # text tfidf

'host_since', # date, number of months

'host_verifications', # string of terms
'amenities', # string of terms

'host_response_rate', # string, strip the % sign
'host_acceptance_rate', # string, strip the % sign
'cleaning_fee', # string, strip the $ sign

'zipcode',  # string, probably one of the most important features
'property_type', # string
'room_type', # string
'accommodates', #int
'bathrooms', # float
'bedrooms', # float
'beds', # float
'bed_type', # string ohe
'guests_included', # int, ohe, anything that is below 1% probably can go
'minimum_nights', # int, ohe, anything that is below 1% probably can go
'maximum_nights', # int, ohe, anything that is below 1% probably can go
'number_of_reviews', # int, ohe, anything that is below 1% probably can go
'review_scores_rating', # float, do nothing for now
'review_scores_accuracy', #float,
'review_scores_cleanliness', #float,
'review_scores_checkin', #float,
'review_scores_communication', #float,
'review_scores_location', #float,
'review_scores_value', #float, 
'instant_bookable', # str t/f, bianry encoding
'cancellation_policy', # str ,4 categories, ohe
'calculated_host_listings_count', #  int, ohe
'reviews_per_month',# float

'price' # target variable
]

print(len(columns))
data = pd.read_csv('/Users/babyhandzzz/Desktop/ELEPH@NT/Datasets/mobidev/data/Train.csv',usecols=columns)
columns = [i for i in columns if i != 'price'] 
print(len(columns))
test_data = pd.read_csv('/Users/babyhandzzz/Desktop/ELEPH@NT/Datasets/mobidev/data/Submission.csv',usecols=columns)

35
34


# TFIDF transfomrmation

In [21]:
def tfidf_columns(df,columns):
    
    list_ = []
    new_df = pd.DataFrame(index=range(df.shape[0]))
    tfidf =TfidfVectorizer(min_df=40, stop_words='english', lowercase=True, max_df=500)
     
    for column in columns:
        df[column].fillna('none',inplace=True) 
        processed_text = tfidf.fit(data[column]) # fitting on original dataset    
        processed_text = tfidf.transform(df[column])
        processed_text = pd.DataFrame(processed_text.toarray(), columns=tfidf.get_feature_names())
        processed_text.columns = [column+'_'+i for i in processed_text.columns]
        
        list_.append(processed_text)
    
    df = pd.concat(list_,axis=1)
    return df

text_columns = [
'summary', # text
'description', # text
'neighborhood_overview', # text
'transit', # text
'host_about'] # text]

# Number of months listed on platform

In [22]:
def host_since(df):
    df = df[['host_since']]
    df['host_since'] = pd.to_datetime(df['host_since'])
    df['now'] =  pd.Timestamp('2020-07-22')
    df = pd.DataFrame((df['now'] - df['host_since'])/np.timedelta64(1, 'M'), columns=['months_active'])
    df['months_active'] = df['months_active'].astype('int')
    return df

# Amenities 

In [23]:
def amenities(df, column):
    df[column] = df[column].str.replace(r'{|"|}|/', '')
    df[column] = df[column].str.replace(' ', '')
    vector = CountVectorizer()
    vector.fit(data['amenities']) # this solutions does not generalize well but it's good enough for now
    series = vector.transform(df[column])
    df = pd.DataFrame(series.toarray(), columns=vector.get_feature_names())
    df.columns = [column+'_'+i for i in df.columns]
    return df

# OHE

In [24]:
def ohe(df,column,prefix,rare=False):
    
    df[column] = df[column].str.replace(" ","")
    df[column] = df[column].str.replace(r'-|&|/', '')
    
    most_frequent_value = data[column].value_counts().idxmax() # most frequent value, has to be from the training dataset
    df[column].fillna(most_frequent_value,inplace=True)
    vectorizer = CountVectorizer()
    vectorizer.fit(data[column]) # training data
    
    if rare==False:
        series = vectorizer.transform(df[column])
        df = pd.DataFrame(series.toarray(),columns=vectorizer.get_feature_names())
        df.columns = [prefix+i for i in df.columns]

    elif rare==True:
        series = df[column].mask(df[column].map(df[column].value_counts(normalize=True)) < 0.01, 'other')
        df = pd.DataFrame(series,columns=[column])
        series = vectorizer.transform(df[column])
        df = pd.DataFrame(series.toarray(),columns=vectorizer.get_feature_names())
        df.columns = [prefix+i for i in df.columns]

    return df

# NaN replacer

In [25]:
def replace_nan(df, column):
    most_frequent_value = df[column].value_counts().idxmax()
    df[column].fillna(most_frequent_value,inplace=True)
    df = df[[column]]
    return df

# PCT(%)

In [26]:
def remove_pct(df, column):
    df = pd.DataFrame(df[column].str.replace("%",""),columns=[column])
    most_frequent_value = df[column].value_counts().idxmax()
    df[column].fillna(most_frequent_value,inplace=True)
    df[column] = df[column].astype(int)
    return df

# Cleaning_fee

In [27]:
def cleaning_fee(df, column):
    df = pd.DataFrame(df[column].str.replace("$",""),columns=[column])
    df.fillna(0,inplace=True)
    df[column] = df[column].astype(float)
    return df

# Target_pre-processing

In [28]:
def pre_proc_price(df,column):
    series = df[column].str.replace(r'$','')
    series = series.str.replace(r',','')
    df = pd.DataFrame(series,columns=[column])
    df['price'] = pd.to_numeric(df['price'])
    return df 

# Scaling numerical features

In [29]:
def scale_num_features(df,num_cols): # unlike most of the other functions here, this one takes a list of columns.
    scaler = StandardScaler()
    scaler.fit(ml_df[num_cols]) # fitting on training dataset
    scaled = scaler.transform(df[num_cols])
    scaled_df = pd.DataFrame(scaled,columns=num_cols)
    df[df.columns[:20]] = scaled_df
    return df

# Packaging it all together (Training_df)

In [30]:
funcs = [
replace_nan(data, 'beds'), # scale
replace_nan(data, 'bathrooms'), # scale
host_since(data), # scale
data[['accommodates']], # scale
data[['guests_included']], # scale
data[['minimum_nights']],# scale
data[['maximum_nights']], # scale
data[['number_of_reviews']], # scale
replace_nan(data, 'review_scores_rating'), # scale
replace_nan(data, 'review_scores_accuracy'), # scale
replace_nan(data, 'review_scores_cleanliness'), # scale
replace_nan(data, 'review_scores_checkin'), # scale
replace_nan(data, 'review_scores_communication'), # scale
replace_nan(data, 'review_scores_location'),# scale
replace_nan(data, 'review_scores_value'), # scale
data[['calculated_host_listings_count']], # scale
replace_nan(data,'reviews_per_month'), # scale
remove_pct(data, 'host_response_rate'),# scale
remove_pct(data, 'host_acceptance_rate'),# scale
cleaning_fee(data, 'cleaning_fee'), # scale
##################################
tfidf_columns(data,text_columns),
amenities(data, 'amenities'),
ohe(data,'zipcode','zip_',False),
ohe(data,'property_type', 'pr_tp_', rare=True),
ohe(data,'room_type','rm_tp_',False),
ohe(data,'bed_type','bd_tp_',True),
ohe(data,'cancellation_policy','canc_pol_',False),
data['instant_bookable'].replace({'f':0,'t':1}),
##################################
pre_proc_price(data,'price')
]


ml_df = pd.concat(funcs, axis=1)
numerical_columns = ml_df.columns[:20]
ml_df = scale_num_features(ml_df, numerical_columns)

# Packaging it all together (Testing_df)

In [31]:
funcs_test = [
replace_nan(test_data, 'beds'), # scale
replace_nan(test_data, 'bathrooms'), # scale
host_since(test_data), # scale
test_data[['accommodates']], # scale
test_data[['guests_included']], # scale
test_data[['minimum_nights']],# scale
test_data[['maximum_nights']], # scale
test_data[['number_of_reviews']], # scale
replace_nan(test_data, 'review_scores_rating'), # scale
replace_nan(test_data, 'review_scores_accuracy'), # scale
replace_nan(test_data, 'review_scores_cleanliness'), # scale
replace_nan(test_data, 'review_scores_checkin'), # scale
replace_nan(test_data, 'review_scores_communication'), # scale
replace_nan(test_data, 'review_scores_location'),# scale
replace_nan(test_data, 'review_scores_value'), # scale
test_data[['calculated_host_listings_count']], # scale
replace_nan(test_data,'reviews_per_month'), # scale
remove_pct(test_data, 'host_response_rate'),# scale
remove_pct(test_data, 'host_acceptance_rate'),# scale
cleaning_fee(test_data, 'cleaning_fee'), # scale
##################################
tfidf_columns(test_data,text_columns),
amenities(test_data, 'amenities'),
ohe(test_data,'zipcode','zip_',False),
ohe(test_data,'property_type', 'pr_tp_', rare=True),
ohe(test_data,'room_type','rm_tp_',False),
ohe(test_data,'bed_type','bd_tp_',True),
ohe(test_data,'cancellation_policy','canc_pol_',False),
test_data['instant_bookable'].replace({'f':0,'t':1})
]

test_df = pd.concat(funcs_test, axis=1)
test_df = scale_num_features(test_df, numerical_columns)

# Modeling

In [32]:
features = ml_df.columns[:-1]
X = ml_df[features]
y = ml_df[['price']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=23)

# Initial candidates

In [33]:
models = [LinearRegression(),GradientBoostingRegressor(),ElasticNet(),Lasso(),DecisionTreeRegressor()]

for model in models:
    model.fit(X_train,y_train)
    print('#######')
    print(model)
    print('Train_score: {}'.format(model.score(X_train,y_train)))
    print('Test_score: {}'.format(model.score(X_test,y_test)))

#######
LinearRegression()
Train_score: 0.9986877332995239
Test_score: -442.89723567638276
#######
GradientBoostingRegressor()
Train_score: 0.8920944194390308
Test_score: 0.3398819280322709
#######
ElasticNet()
Train_score: 0.34698171501710595
Test_score: 0.35739312949075075
#######
Lasso()
Train_score: 0.38796424692733034
Test_score: 0.3753925166717226
#######
DecisionTreeRegressor()
Train_score: 1.0
Test_score: -1.0303513219524203


# Narrowing down to Lasso and Gradient Boosting

In [16]:
models = [Lasso(alpha=2,max_iter=1000),GradientBoostingRegressor(n_estimators=2000, min_samples_leaf=20)]

for model in models:
    model.fit(X_train,y_train)
    print('#######')
    print(model)
    print('Train_score: {}'.format(model.score(X_train,y_train)))
    print('Test_score: {}'.format(model.score(X_test,y_test)))

NameError: name 'X_train' is not defined

In [17]:
predictions = pd.Series(models[1].predict(X))
sanity_check = pd.DataFrame({'Predictions':predictions,"Actual Price": y['price']})
sanity_check['difference'] = sanity_check['Predictions'] - sanity_check['Actual Price']
sanity_check

NameError: name 'X' is not defined

# Top Impact Factors 

In [18]:
feature_importance = pd.DataFrame({'importance':models[1].feature_importances_, 'feature':X.columns})
feature_importance.sort_values('importance',ascending=False)[:20]

NotFittedError: This GradientBoostingRegressor instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.